In [1]:
import numpy as np

subject, image, label 파일들 txt파일로 만들어서 저장

In [4]:
lst = []

for idx in range(11965):
    # file = open('subject.txt','w')
    data = np.load(f"datasets/eegdataset/subject/{idx}.npy")
    data = str(data)
    lst.append(data)
    # w = file.write(data)
print(lst)
print(len(lst))

In [ ]:
lst = []

for idx in range(11965):
    data = np.load(f"/Data/summer24/DreamDiffusion/datasets/eegdataset/image/{idx}.npy")
    data = str(data)
    lst.append(data)
    # w = file.write(data)
print(lst)
print(len(lst))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '15

In [ ]:
file_name = './image.txt'

with open(file_name, 'w+') as file:
    file.write('\n'.join(lst))

In [ ]:
data = np.load("/Data/summer24/DreamDiffusion/datasets/eegdataset/eeg/1555.npy")
print(data)
print(data.shape)

[[-0.00697945 -0.06386714 -0.1064683  ...  0.11467142  0.05829991
   0.01750932]
 [ 0.00227203  0.0244599   0.06157871 ...  0.17750017  0.09323435
   0.0141344 ]
 [ 0.00980269 -0.04240372 -0.10028582 ... -0.15474936 -0.07398553
  -0.00515919]
 ...
 [ 0.02253668  0.2698289   0.5181813  ... -0.07486256 -0.05740875
  -0.01423337]
 [-0.004018    0.00988592  0.02385702 ... -0.00490741 -0.0060739
  -0.00551859]
 [ 0.0118606   0.2280959   0.44277313 ... -0.06660569 -0.05307152
  -0.01054234]]
(128, 497)


다 128 channel인지 확인 -> 다 128 channel 인데, fs*t 는 달랐음

In [ ]:
for idx in range(11965):
    data = np.load(f"/Data/summer24/DreamDiffusion/datasets/eegdataset/eeg/{idx}.npy")
    if data.shape[0] != 128:
        print(idx)


데이터셋에서 train에 사용한 대로 test에도 사용했는지 확인

In [ ]:
import torch
split_file_single = torch.load('/Data/summer24/DreamDiffusion/datasets/block_splits_by_image_single.pth')
print(split_file_single)
print(len(split_file_single["splits"][0]["train"]))
print(len(split_file_single["splits"][0]["val"]))
print(len(split_file_single["splits"][0]["test"]))

{'splits': [{'train': [0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 27, 29, 33, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 51, 52, 54, 55, 57, 58, 60, 61, 62, 63, 66, 68, 69, 70, 71, 72, 73, 74, 77, 78, 79, 83, 84, 87, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 110, 112, 113, 115, 116, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 135, 137, 138, 140, 141, 142, 143, 145, 146, 147, 149, 150, 151, 152, 153, 156, 157, 159, 161, 163, 164, 166, 168, 169, 173, 174, 175, 176, 177, 179, 180, 182, 185, 186, 187, 188, 189, 190, 191, 194, 195, 199, 200, 202, 208, 209, 211, 212, 213, 215, 217, 220, 222, 223, 224, 226, 227, 228, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 241, 242, 243, 244, 247, 248, 250, 252, 253, 255, 256, 257, 258, 259, 260, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 275, 278, 279, 280, 281, 282, 284, 285, 286, 287, 288, 291, 292, 293, 295, 301, 302, 303, 305, 307, 

In [57]:
import torch
split_file_all = torch.load('/Data/summer24/DreamDiffusion/datasets/block_splits_by_image_all.pth')
print(split_file_all)

print(len(split_file_all["splits"][0]["train"]))
print(len(split_file_all["splits"][0]["val"]))
print(len(split_file_all["splits"][0]["test"]))

{'splits': [{'train': [0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 27, 29, 33, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 51, 52, 54, 55, 57, 58, 60, 61, 62, 63, 66, 68, 69, 70, 71, 72, 73, 74, 77, 78, 79, 83, 84, 87, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 110, 112, 113, 115, 116, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 135, 137, 138, 140, 141, 142, 143, 145, 146, 147, 149, 150, 151, 152, 153, 156, 157, 159, 161, 163, 164, 166, 168, 169, 173, 174, 175, 176, 177, 179, 180, 182, 185, 186, 187, 188, 189, 190, 191, 194, 195, 199, 200, 202, 208, 209, 211, 212, 213, 215, 217, 220, 222, 223, 224, 226, 227, 228, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 241, 242, 243, 244, 247, 248, 250, 252, 253, 255, 256, 257, 258, 259, 260, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 275, 278, 279, 280, 281, 282, 284, 285, 286, 287, 288, 291, 292, 293, 295, 301, 302, 303, 305, 307, 

In [16]:
import torch
data = torch.load('/Data/summer24/DreamDiffusion/datasets/eeg_5_95_std.pth')

In [21]:
data['dataset'][5555]['subject']

6

In [28]:
from dataset import identity, Dataset
from scipy.interpolate import interp1d
from transformers import AutoProcessor
import os
from PIL import Image

class EEGDataset(Dataset):
    
    # Constructor
    def __init__(self, eeg_signals_path, image_transform=identity, subject = 0):
        # Load EEG signals
        loaded = torch.load(eeg_signals_path)
        if subject!=0:
            self.data = [loaded['dataset'][i] for i in range(len(loaded['dataset']) ) if loaded['dataset'][i]['subject']==subject]
        else:
            self.data = loaded['dataset']        
        self.labels = loaded["labels"]
        self.images = loaded["images"]
        self.imagenet = 'datasets/imageNet_images'
        self.image_transform = image_transform
        self.num_voxels = 440
        self.data_len = 512
        # Compute size
        self.size = len(self.data)
        self.processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")

    # Get size
    def __len__(self):
        return self.size

    # Get item
    def __getitem__(self, i):
        # Process EEG
        # print(self.data[i])
        eeg = self.data[i]["eeg"].float().t()

        eeg = eeg[20:460,:]
        ##### 2023 2 13 add preprocess and transpose
        eeg = np.array(eeg.transpose(0,1))
        x = np.linspace(0, 1, eeg.shape[-1])
        x2 = np.linspace(0, 1, self.data_len)
        f = interp1d(x, eeg)
        eeg = f(x2)
        eeg = torch.from_numpy(eeg).float()
        ##### 2023 2 13 add preprocess
        label = torch.tensor(self.data[i]["label"]).long()

        # Get label
        image_name = self.images[self.data[i]["image"]]
        image_path = os.path.join(self.imagenet, image_name.split('_')[0], image_name+'.JPEG')
        # print(image_path)
        image_raw = Image.open(image_path).convert('RGB') 
        
        image = np.array(image_raw) / 255.0
        image_raw = self.processor(images=image_raw, return_tensors="pt")
        image_raw['pixel_values'] = image_raw['pixel_values'].squeeze(0)


        return {'eeg': eeg, 'label': label, 'image': self.image_transform(image), 'image_raw': image_raw}

In [29]:
from dataset import EEGDataset, create_EEG_dataset
from config import Config_MBM_EEG
eeg_signals_path = "/Data/summer24/DreamDiffusion/datasets/eeg_5_95_std.pth"
splits_path = "/Data/summer24/DreamDiffusion/datasets/block_splits_by_image_all.pth"
dataset_train, dataset_val = create_EEG_dataset(eeg_signals_path, splits_path, subject = 0)

In [58]:
split_file_all

{'splits': [{'train': [0,
    3,
    4,
    5,
    6,
    8,
    9,
    10,
    11,
    12,
    13,
    15,
    16,
    17,
    18,
    19,
    21,
    22,
    23,
    24,
    25,
    27,
    29,
    33,
    35,
    36,
    37,
    38,
    39,
    42,
    43,
    44,
    45,
    46,
    47,
    48,
    51,
    52,
    54,
    55,
    57,
    58,
    60,
    61,
    62,
    63,
    66,
    68,
    69,
    70,
    71,
    72,
    73,
    74,
    77,
    78,
    79,
    83,
    84,
    87,
    88,
    89,
    90,
    91,
    93,
    94,
    95,
    96,
    97,
    98,
    99,
    100,
    101,
    103,
    104,
    105,
    106,
    107,
    108,
    110,
    112,
    113,
    115,
    116,
    118,
    119,
    120,
    121,
    122,
    123,
    124,
    125,
    127,
    128,
    129,
    130,
    131,
    135,
    137,
    138,
    140,
    141,
    142,
    143,
    145,
    146,
    147,
    149,
    150,
    151,
    152,
    153,
    156,
    157,
    159,
    161,
    163,
    16

In [105]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import BatchSampler
import torchvision.transforms as transforms
import torch
import numpy as np
from PIL import Image
import os

def load_data(eeg_path, img_path, splits_path, batch_size=16):
    """
    Args:
        is_inception: True | False
    """
    loaded_eeg = torch.load(eeg_path)
    loaded_splits = torch.load(splits_path)['splits']
    
    train_dataset = EEGDataset(img_path, loaded_eeg, loaded_splits, mode="train")
    val_dataset = EEGDataset(img_path, loaded_eeg, loaded_splits, mode="val")
    test_dataset = EEGDataset(img_path, loaded_eeg, loaded_splits, mode="test")


    # kwargs = {'num_workers': 4, 'pin_memory': True} if device else {}
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, )
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True )
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    return train_dataloader, val_dataloader, test_dataloader

class EEGDataset(Dataset):
    """
    Train: For each sample (anchor) randomly chooses a positive and negative samples
    Test: Creates fixed triplets for testing
    """
    def __init__(self, img_dir_path, loaded_eeg, loaded_splits, mode="train", transform=None):
        """
        Args:
            img_dir_path: directory path of imagenet images,
            loaded_eeg: eeg dataset loaded from torch.load(),
            loaded_splits: cross-validation splits loaded from torch.load(),
        All arrays and data are returned as torch Tensors
        """
        self.mode = mode
        self.transform = transform
        self.img_dir_path = img_dir_path
        self.splits = loaded_splits
        dataset, classes, img_filenames = [loaded_eeg[k] for k in ['dataset', 'labels', 'images']]
        self.classes = classes
        self.img_filenames = img_filenames

        self.eeg_dataset = dataset
        """We use only split 0, no cross-validation"""
        self.split_chosen = loaded_splits[0]
        self.split_train = self.split_chosen['train']
        self.split_val = self.split_chosen['val']
        self.split_test = self.split_chosen['test']

        if self.mode == "train":
            self.labels = [self.eeg_dataset[sample_idx]['label'] for sample_idx in self.split_train]
        elif self.mode == "val":
            self.labels = [self.eeg_dataset[sample_idx]['label'] for sample_idx in self.split_val]
        elif self.mode == "test":
            self.labels = [self.eeg_dataset[sample_idx]['label'] for sample_idx in self.split_test]
        else:
            raise ValueError()

        self.labels = torch.tensor(self.labels)
    def __getitem__(self, index):
        """
        Return: (eeg, img), []
            - eeg: Tensor()
            - image: Tensor()
        """
        if self.mode == "train":
            dataset_idx = self.split_train[index]
        elif self.mode == "val":
            dataset_idx = self.split_val[index]
        elif self.mode == "test":
            dataset_idx = self.split_test[index]
        else:
            raise ValueError()
        eeg,_, label = [self.eeg_dataset[dataset_idx][key] for key in ['eeg', 'image', 'label']]

        return eeg, label

    def __len__(self):
        if self.mode == "train":
            return len(self.split_train)
        elif self.mode == "val":
            return len(self.split_val)
        elif self.mode == "test":
            return len(self.split_test)
        else:
            raise ValueError()
        


In [106]:
eeg_path = "/Data/summer24/DreamDiffusion/datasets/eeg_5_95_std.pth"
img_path = "/Data/summer24/DreamDiffusion/datasets/imageNet_images"
splits_path = "/Data/summer24/DreamDiffusion/datasets/block_splits_by_image_all.pth"
train_dataloader, val_dataloader, test_dataloader = load_data(eeg_path,img_path,splits_path)

In [107]:
loaded_eeg = torch.load(eeg_path)
loaded_splits = torch.load(splits_path)['splits']

train_dataset = EEGDataset(img_path, loaded_eeg, loaded_splits, mode="train")

In [102]:
len(train_dataset)

7970

In [104]:
lst = []
for idx in range (len(train_dataset)):
    print(train_dataset[idx][1])

10
10
30
25
18
3
8
11
18
28
38
20
3
28
23
0
34
20
23
39
0
34
21
39
6
26
20
1
27
37
19
9
12
18
25
27
34
35
8
29
12
17
37
27
25
32
35
5
8
20
31
25
31
37
6
37
32
20
30
23
18
2
3
19
26
20
27
27
18
39
33
36
21
5
8
39
24
34
13
36
8
39
30
1
19
37
11
24
36
12
7
29
37
6
30
33
5
27
2
4
30
7
8
12
33
24
30
32
5
17
23
33
28
34
7
14
18
0
24
39
23
27
28
30
26
17
15
30
38
22
23
11
28
12
20
34
26
4
36
0
5
34
25
12
9
23
10
35
5
0
18
29
34
29
28
17
8
17
22
15
21
30
16
23
3
13
22
1
15
33
30
7
39
8
7
30
38
24
3
6
26
1
28
36
21
32
7
9
30
36
35
8
28
38
36
4
32
39
30
4
26
3
9
39
30
29
22
22
22
35
38
3
22
8
6
30
13
1
1
11
13
11
27
8
1
14
8
24
23
8
17
34
19
6
33
6
14
9
27
34
1
34
3
3
1
16
3
1
12
27
36
33
1
38
33
0
9
3
3
1
20
39
11
24
20
14
7
32
14
35
33
9
17
8
39
29
7
39
17
35
28
34
8
18
2
16
20
31
39
8
36
5
9
29
28
25
13
20
2
16
21
27
34
5
35
0
4
22
23
15
23
14
35
31
8
6
15
12
2
4
2
28
16
20
0
35
13
10
19
8
18
8
15
27
4
14
34
23
26
29
22
18
37
20
33
36
12
31
36
32
35
20
15
37
19
11
10
16
16
1
18
8
20
18
19
23


In [94]:
train_dataset[1]

(tensor([[ 0.0076,  0.0605,  0.1214,  ...,  0.1027,  0.0541,  0.0113],
         [ 0.0049,  0.0806,  0.1632,  ...,  0.0397,  0.0348,  0.0237],
         [-0.0148,  0.0531,  0.1162,  ...,  0.1000,  0.0386, -0.0223],
         ...,
         [-0.0229,  0.0678,  0.1807,  ..., -0.5030, -0.2601,  0.0196],
         [-0.0062,  0.0032,  0.0129,  ..., -0.0317, -0.0186, -0.0039],
         [-0.0179,  0.1185,  0.2597,  ..., -0.4428, -0.2308,  0.0150]]),
 10)

In [74]:
train_dataset

NameError: name 'train_dataset' is not defined

In [70]:
train_dataloader

In [73]:
next(iter(train_dataloader))

RuntimeError: stack expects each tensor to be equal size, but got [128, 521] at entry 0 and [128, 504] at entry 1

In [109]:
from dataset import  create_EEG_dataset
from config import Config_Generative_Model
from einops import rearrange

class random_crop:
    def __init__(self, size, p):
        self.size = size
        self.p = p
    def __call__(self, img):
        if torch.rand(1) < self.p:
            return transforms.RandomCrop(size=(self.size, self.size))(img)
        return img

def channel_last(img):
        if img.shape[-1] == 3:
            return img
        return rearrange(img, 'c h w -> h w c')

def normalize(img):
    if img.shape[-1] == 3:
        img = rearrange(img, 'h w c -> c h w')
    img = torch.tensor(img)
    img = img * 2.0 - 1.0 # to -1 ~ 1
    return img


config = Config_Generative_Model()

crop_pix = int(config.crop_ratio*config.img_size)

img_transform_train = transforms.Compose([
    normalize,

    transforms.Resize((512, 512)),
    random_crop(config.img_size-crop_pix, p=0.5),

    transforms.Resize((512, 512)),
    channel_last
])
img_transform_test = transforms.Compose([
    normalize, 

    transforms.Resize((512, 512)),
    channel_last
    ])


In [114]:
eeg_signals_path = "/Data/summer24/DreamDiffusion/datasets/eeg_5_95_std.pth"
splits_path = "/Data/summer24/DreamDiffusion/datasets/block_splits_by_image_single.pth"


eeg_latents_dataset_train, eeg_latents_dataset_test = create_EEG_dataset(eeg_signals_path, splits_path, 
                image_transform=[img_transform_train, img_transform_test], subject = config.subject)

In [115]:
dir(eeg_latents_dataset_train)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'data_len',
 'dataset',
 'num_voxels',
 'size',
 'split_idx']

In [118]:
dir(eeg_latents_dataset_train.dataset)

['__add__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_is_protocol',
 'data',
 'data_len',
 'image_transform',
 'imagenet',
 'images',
 'labels',
 'num_voxels',
 'processor',
 'size']

In [119]:
eeg_latents_dataset_train.dataset.images

['n02951358_31190',
 'n03452741_16744',
 'n04069434_10318',
 'n02951358_34807',
 'n03452741_5499',
 'n03376595_247',
 'n02906734_14010',
 'n03452741_2982',
 'n02607072_5873',
 'n02510455_96034',
 'n02281787_4182',
 'n02906734_508',
 'n03272010_4472',
 'n04044716_691',
 'n03272562_1177',
 'n03773504_14125',
 'n02607072_3895',
 'n03272010_6902',
 'n03709823_451',
 'n02389026_2743',
 'n04044716_8814',
 'n03197337_1807',
 'n03773504_2986',
 'n03709823_25676',
 'n02124075_9271',
 'n02389026_27266',
 'n03180011_9576',
 'n03197337_1925',
 'n02281787_4713',
 'n02492035_1742',
 'n03773504_6291',
 'n02951358_4819',
 'n03272562_4339',
 'n02124075_4538',
 'n04086273_29346',
 'n03792782_1424',
 'n03877472_53538',
 'n03773504_2899',
 'n03888257_5543',
 'n03775071_11195',
 'n03452741_3088',
 'n02504458_7221',
 'n03272562_2929',
 'n07873807_6649',
 'n11939491_8940',
 'n02106662_2328',
 'n02906734_6563',
 'n03376595_5336',
 'n03775071_4141',
 'n03297495_7249',
 'n04120489_4154',
 'n03197337_544',
 'n02

In [120]:
eeg_latents_dataset_train.dataset.labels

['n02389026',
 'n03888257',
 'n03584829',
 'n02607072',
 'n03297495',
 'n03063599',
 'n03792782',
 'n04086273',
 'n02510455',
 'n11939491',
 'n02951358',
 'n02281787',
 'n02106662',
 'n04120489',
 'n03590841',
 'n02992529',
 'n03445777',
 'n03180011',
 'n02906734',
 'n07873807',
 'n03773504',
 'n02492035',
 'n03982430',
 'n03709823',
 'n03100240',
 'n03376595',
 'n03877472',
 'n03775071',
 'n03272010',
 'n04069434',
 'n03452741',
 'n03792972',
 'n07753592',
 'n13054560',
 'n03197337',
 'n02504458',
 'n02690373',
 'n03272562',
 'n04044716',
 'n02124075']

In [123]:
eeg_latents_dataset_train.dataset.data

[{'eeg': tensor([[-8.6142e-02, -2.9463e-01, -6.2266e-01,  ..., -2.0556e-01,
           -2.0201e-01, -2.2041e-01],
          [-4.6275e-02, -1.5711e-01, -3.3941e-01,  ..., -4.8833e-02,
           -7.8179e-02, -1.2174e-01],
          [ 2.5091e-02,  1.3313e-01,  2.6716e-01,  ...,  8.9221e-02,
            6.9278e-02,  5.2495e-02],
          ...,
          [ 8.0494e-02,  3.2728e-01,  4.4147e-01,  ...,  1.4659e+00,
            7.8212e-01,  2.5284e-03],
          [-1.2392e-03,  1.0254e-02,  1.4863e-02,  ...,  7.1831e-02,
            3.5992e-02, -5.0048e-03],
          [ 5.3350e-02,  2.9741e-01,  4.4446e-01,  ...,  8.7492e-01,
            4.6427e-01, -4.8223e-03]]),
  'image': 0,
  'label': 10,
  'subject': 6},
 {'eeg': tensor([[-0.0595, -0.2462, -0.5524,  ...,  2.3339,  1.2344,  0.1786],
          [-0.0297, -0.1574, -0.3423,  ...,  1.4544,  0.7648,  0.0985],
          [ 0.0175,  0.0805,  0.1608,  ..., -0.4176, -0.2216, -0.0385],
          ...,
          [-0.0880,  0.4591,  0.6931,  ...,  0.848

In [127]:
df = eeg_latents_dataset_train.dataset.data

In [131]:
lst = []
for i in range(100):
    lst.append(df[i]["subject"])
print(lst)

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]


In [130]:
df[0]["subject"]

6